# Extracting and Loading the CSV

In [ ]:
# importing libraries

import os
import sys
import pandas as pd

In [ ]:
# loading scripts

sys.path.append(os.path.abspath("../scripts/python/"))
from loader import Loader
from extractor import Extractor

loader = Loader()
extractor = Extractor()

## Extract Data

In [ ]:
# load csv into a dataframe
data = extractor.load_csv("../data/sample.csv")
data.head()

In [ ]:
# restructure data frame.
new_df = extractor.restructure(data)
new_df.head()

## Load Into Warehouse

In [ ]:
# create connection with wareouse
connection, cursor = loader.connect_to_db("localhost", "root", "localhost")

In [ ]:
loader.create_db(cursor, "warehouse")

In [ ]:
conn, cur = loader.connect_to_db("localhost", "root", "localhost", "warehouse")

In [ ]:
loader.create_table(cur, "../scripts/sql/source_table.sql", "warehouse")

In [ ]:
loader.insert_into_warehouse(cur, conn, "warehouse", new_df, "source")

In [ ]:
loader.close_connection(cursor=cursor, connection=connection)